# Code Sandbox

A place to test new code and functions

In [1]:
# import packages and set working directory
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from math import exp
import seaborn as sns

#os.chdir('../maize-Toff')

# import objects
from farm import Climate
from farm import Soil
from farm import Crop
from farm import CropModel
from farm.functions import *

In [8]:
climate = Climate(station='OL JOGI FARM')

soil = Soil(texture='clay loam')
crop = Crop(soil=soil, lgp=180)
soil.set_nZr(crop)

model = CropModel(crop=crop, climate=climate, soil=soil)
model.run()
o = model.output() # This is the correct length
o.head(25)
o_ = o.loc[o['dos'] >= 1]
o_.head(20)

,kc,LAI,stress,R,s,E,ET,T,L,dsdt,dos,doy
21,0.3,0.75,1.000000,0.000000,0.486038,0.171633,0.171633,0.000000,0.0,-0.171633,1.0,100
22,0.3,0.75,1.000000,13.716494,0.485317,0.168829,0.168829,0.000000,0.0,13.547665,2.0,101
23,0.3,0.75,0.904480,0.000000,0.542240,0.432244,0.490994,0.058750,0.0,-0.490994,3.0,102
24,0.3,0.75,0.920244,0.000000,0.540177,0.421348,0.470195,0.048848,0.0,-0.470195,4.0,103
25,0.3,0.75,0.935468,0.000000,0.538201,0.411001,0.450365,0.039365,0.0,-0.450365,5.0,104
26,0.3,0.75,0.950167,0.422892,0.536309,0.401171,0.431452,0.030282,0.0,-0.008560,6.0,105
27,0.3,0.75,0.950448,0.000000,0.536273,0.400985,0.431094,0.030109,0.0,-0.431094,7.0,106
28,0.3,0.75,0.964627,0.000000,0.534461,0.391651,0.413066,0.021415,0.0,-0.413066,8.0,107
29,0.3,0.75,0.978312,0.000000,0.532726,0.382777,0.395861,0.013084,0.0,-0.395861,9.0,108
30,0.3,0.75,0.991518,17.124354,0.531063,0.374337,0.379437,0.005100,0.0,16.744917,10.0,109


In [34]:
# playing around with dynamic stress

KPAR_CROP = 0.5
RPAR_CROP = 2
TGRW_CROP = 180
QPAR_CROP = 2

INVL_SIMU = 1 # actually really not sure about this

SWLT_SOIL = crop.sw
SSTR_SOIL = crop.s_star

## Define Function
def calc_dstress(s, stress): #self,  add this
    '''Calculates dyamic stress (theta) which is a measure of total water stress during the growing season
    as proposed in Porporato et al. (2001).
    
    Usage: calc_dstress(s, stress):
        s = relative saturation [0-1]
        stress = static stress [0-1]
        
    Arguments:
    average static stress = not sure about this: mstr_memb
    stress point T = average duration of an excursion below stress point
    n_star = number of excursions during the growing season
    T_seas = length of the growing season: TGRW_CROP
    k = fraction of growing season before crop fails = KPAR_CROP
    r = effect of number of excurs below strss point = RPAR_CROP
    '''
    # Calculate average static stress
    if len(stress) > 0:
        # It's annoying but need to subset the time series so it's just the growing season
        # Subset the growing period and get avg soil moisture
        start = 21 
        end = start + TGRW_CROP
        stress_subset = stress[start:end]
        mstr_memb = np.mean(stress_subset)
    else:
        mstr_memb = 0.

    # Calculate Crossing Parameters
    indx_memb = np.where(s >= SSTR_SOIL)
    ccrs_memb = np.diff(np.append(0, np.append(indx_memb, INVL_SIMU * TGRW_CROP + 1))) - 1
    ccrs_memb = ccrs_memb[ccrs_memb > 0]
    ncrs_memb = len(ccrs_memb)  # dim
    if ncrs_memb > 0:
        mcrs_memb = np.mean(ccrs_memb) / INVL_SIMU # days
    else:
        mcrs_memb = 0.
        
    # Calculate dynamic stress
    dstr_memb = ((mstr_memb * mcrs_memb) / (KPAR_CROP * TGRW_CROP))**(ncrs_memb**-RPAR_CROP)
    if dstr_memb > 1.:
        dstr_memb = 1.

    return(dstr_memb)


calc_dstress(s = o.s, stress = o.stress)

# convert this into yield
#4260*(calc_dstress(memb = o.s, stress = o.stress))


0.5618054554391123

In [10]:
# Use Drew's code to write a test

SWLT_SOIL = crop.sw
SSTR_SOIL = crop.s_star
QPAR_CROP = 2

    # i added this in    
# Calculate Average Static Stress -- I think I already do this myself and can just import this average value
sstr_memb = (SSTR_SOIL - o.s) / (SSTR_SOIL - SWLT_SOIL)  # dim
sstr_memb = sstr_memb[sstr_memb > 0.]
sstr_memb[sstr_memb > 1.] = 1.
if len(sstr_memb) > 0:
    mstr_memb = np.mean(sstr_memb**QPAR_CROP)  # dim
else:
    mstr_memb = 0.  # dim
    
    # Can write a test using Drew's code
    # above is Drew's code. If I just want to use the stress function I already calculcated, do:
    
mstr_memb

0.19388840256942838

In [4]:
average_soil_moisture(model, n_sims=1000, doy=90)

(59    0.555472
 dtype: float64,
 59    0.077929
 dtype: float64)